# Объединение таблиц

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
import sys, os
import pathlib
from pathlib import Path
from pydantic import ValidationError
from olist_churn_prediction.paths import SRC_DIR, PROCESSED_DIR, INTERIM_DIR

In [2]:
sys.path.append(str(SRC_DIR))

In [3]:
from olist_churn_prediction import feature_processing, feature_engineering
from olist_churn_prediction.schemas_interim import MainClassifiedSchema
from olist_churn_prediction.paths import RAW_DIR

ImportError: cannot import name 'MainClassifiedSchema' from 'olist_churn_prediction.schemas_interim' (C:\Users\Вячеслав\Documents\olist_churn\olist-ml-churn-prediction\src\olist_churn_prediction\schemas_interim.py)

In [7]:
display(df.head())

,order_status,order_products_value,order_freight_value,order_items_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,customer_city,customer_state,...,votes_low_quality,votes_return,votes_not_as_anounced,votes_partial_delivery,votes_other_delivery,votes_other_order,votes_satisfied,most_voted_subclass,most_voted_class,product_category_name_english
0,delivered,89.989998,14.380000,1,2017-08-30 11:41:01,2017-08-30 11:55:08.970352,2017-09-21,2017-09-08 20:35:27.276847,belo_horizonte,mg,...,0,0,0,0,0,0,3,satisfeito,satisfeito_com_pedido,health_beauty
1,delivered,69.000000,15.230000,1,2017-09-26 09:13:36,2017-09-26 09:28:10.922048,2017-10-24,2017-09-29 21:13:04.984841,pocos_de_caldas,mg,...,0,0,0,0,0,0,0,antes_prazo,satisfeito_com_pedido,toys
2,delivered,99.800003,15.860000,2,2018-01-15 15:50:42,2018-01-17 07:29:56.419769,2018-02-05,2018-01-23 17:51:31.134866,sao_jose_dos_campos,sp,...,0,0,0,3,0,0,0,entrega_parcial,problemas_de_entrega,garden_tools
3,delivered,87.000000,12.740000,1,2018-02-04 11:16:42,2018-02-06 05:31:50.990164,2018-03-13,2018-02-20 19:38:06.633080,ribeirao_preto,sp,...,0,0,0,0,0,0,0,atrasado,problemas_de_entrega,computers_accessories
4,delivered,99.900002,17.950001,1,2017-12-07 11:58:42,2017-12-08 02:36:49.587515,2018-01-03,2017-12-19 22:33:18.952512,rio_de_janeiro,rj,...,0,0,0,3,0,0,0,entrega_parcial,problemas_de_entrega,bed_bath_table


In [8]:
df.dtypes

order_status                          string[python]
order_products_value                         float32
order_freight_value                          float32
order_items_qty                                Int64
order_purchase_timestamp              datetime64[ns]
order_aproved_at                      datetime64[ns]
order_estimated_delivery_date         datetime64[ns]
order_delivered_customer_date         datetime64[ns]
customer_city                         string[python]
customer_state                        string[python]
customer_zip_code_prefix              string[python]
product_name_lenght                            Int64
product_description_lenght                     Int64
product_photos_qty                             Int64
review_score                                   Int64
review_comment_message                string[python]
review_creation_date                  datetime64[ns]
review_answer_timestamp          datetime64[ns, UTC]
votes_before_estimate                         

In [12]:
df = pd.read_parquet(INTERIM_DIR / "classified_data_interim.parquet", engine="pyarrow")

for i, row in df.head(5).iterrows():
    try:
        MainClassifiedSchema.model_validate(row.to_dict())
    except ValidationError as e:
        print(f"\nRow {i}:\n{e}\n")


Row 0:
3 validation errors for MainClassifiedSchema
id
  Field required [type=missing, input_value={'order_status': 'deliver...glish': 'health_beauty'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
order_sellers_qty
  Field required [type=missing, input_value={'order_status': 'deliver...glish': 'health_beauty'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
product_category_name
  Field required [type=missing, input_value={'order_status': 'deliver...glish': 'health_beauty'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing


Row 1:
3 validation errors for MainClassifiedSchema
id
  Field required [type=missing, input_value={'order_status': 'deliver...y_name_english': 'toys'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
order_sellers_qty
  Field required [type=missing, input_value={'order_status': 

In [13]:
classified_data_interim = feature_processing.load_data(INTERIM_DIR / "classified_data_interim.parquet",
                      schema = MainClassifiedSchema,
                      validate = True) # в проде можно False

ValueError: Schema validation failed for rows: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] (total=3467)